# Definitions

In [1]:
from pathlib import Path
import pickle

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import lightgbm as lgb
from category_encoders import OrdinalEncoder
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import roc_curve, roc_auc_score
import category_encoders as ce
import optuna

In [2]:
data_path = Path('.', 'data')

In [3]:
RANDOM_STATE = 648

In [4]:
target_name = 'dep_delayed_15min'

# Data

In [5]:
train = pd.read_csv(data_path.joinpath('flight_delays_train.csv'))
test = pd.read_csv(data_path.joinpath('flight_delays_test.csv'))

## Route

In [6]:
train['route'] = train.Origin + train.Dest

In [7]:
test['route'] = test.Origin + test.Dest

In [8]:
train[target_name] = train[target_name].map({"Y": 1, "N": 0})

In [9]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 10 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   Month              100000 non-null  object
 1   DayofMonth         100000 non-null  object
 2   DayOfWeek          100000 non-null  object
 3   DepTime            100000 non-null  int64 
 4   UniqueCarrier      100000 non-null  object
 5   Origin             100000 non-null  object
 6   Dest               100000 non-null  object
 7   Distance           100000 non-null  int64 
 8   dep_delayed_15min  100000 non-null  int64 
 9   route              100000 non-null  object
dtypes: int64(3), object(7)
memory usage: 7.6+ MB


## Split Data

In [10]:
X, y = train.drop(target_name, axis=1), train[target_name]

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=RANDOM_STATE)

# Preproc

## timestamp decoding

In [12]:
def get_ts(df):
    df['hour'] = df.DepTime // 100
    df['minute'] = df.DepTime % 100
    df['d_month'] = df.DayofMonth.str[2:].astype(int)
    df['d_week'] = df.DayOfWeek.str[2:].astype(int)
    df['month'] = df.Month.str[2:].astype(int)
    
    return df

## timestamp encoding

In [13]:
def ts_features(df):
    X_ = df[df.DepTime < 2400]
    dep_dt = pd.to_datetime('2021-' +
                                  X_.month.astype(str).str.zfill(2) + '-' +
                                  X_.d_month.astype(str).str.zfill(2) + '-' +
                                  X_.hour.astype(str).str.zfill(2) + '-' +
                                  X_.minute.astype(str).str.zfill(2) + '-00',
                                  format='%Y-%m-%d-%H-%M-%S')
    dep_td = dep_dt - pd.to_datetime('2021-01-01')
    df['days_since_ny'] = dep_td.dt.days
    df['secs_since_ny'] = dep_td.dt.seconds
    
    return df

## Category encoding

In [14]:
cat_cols = X_train.select_dtypes(include=[object, 'category']).columns

In [15]:
cat_cols

Index(['Month', 'DayofMonth', 'DayOfWeek', 'UniqueCarrier', 'Origin', 'Dest',
       'route'],
      dtype='object')

### Target encoding

In [16]:
encoders_params = dict(cols=cat_cols,
                        handle_missing='return_nan',
                        handle_unknown='return_nan')

In [17]:
target_encoders = {'js': ce.JamesSteinEncoder(**encoders_params),
                   'loo': ce.LeaveOneOutEncoder(**encoders_params),
                   'target': ce.TargetEncoder(**encoders_params),
                   'woe': ce.WOEEncoder(**encoders_params)}

In [18]:
%%time
for name, encoder in target_encoders.items():
    encoder.fit(X_train[cat_cols], y_train)
    print(f'{name} finished!')

/Users/aapiskotin/opt/anaconda3/lib/python3.8/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


js finished!
loo finished!


/Users/aapiskotin/opt/anaconda3/lib/python3.8/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


target finished!


/Users/aapiskotin/opt/anaconda3/lib/python3.8/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


woe finished!
CPU times: user 1.62 s, sys: 90.9 ms, total: 1.71 s
Wall time: 1.87 s


In [19]:
enc = OrdinalEncoder(cols=cat_cols,
                     handle_missing='return_nan',
                     handle_unknown='return_nan')

In [20]:
%%time
enc.fit(X_train[cat_cols])
print('Done')

/Users/aapiskotin/opt/anaconda3/lib/python3.8/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


Done
CPU times: user 210 ms, sys: 16.4 ms, total: 227 ms
Wall time: 256 ms


## Sin/cos ts processing

In [21]:
t_limits = {'minute': (0, 59),
             'hour': (0, 23),
             'd_week': (1, 7),
             'd_month': (1, 31),
             'month': (1, 12),
             'days_since_ny': (0, 364)}

In [22]:
def sin_cos(df):
    for col_name, limits in t_limits.items():
        max_val = limits[1]
        min_val = limits[0]
        if col_name == 'hour':
            orig_vals = df[col_name].clip(upper=max_val)
        else:
            orig_vals = df[col_name]
        df[col_name + '_sin'] = np.sin(2*np.pi/(max_val - min_val)*orig_vals)
        df[col_name + '_cos'] = np.cos(2*np.pi/(max_val - min_val)*orig_vals)
        
    return df

## Sum up prepocessing

In [23]:
def preproc(df, enc):
    df = get_ts(df)
    df = ts_features(df)
    df = sin_cos(df)
    
#     for name, encoder in target_encoders.items():
#         df[cat_cols + f'_{name}'] = encoder.transform(df[cat_cols])
    df[cat_cols + '_enc'] = enc.transform(df[cat_cols])
#     df[cat_cols] = df[cat_cols].astype('category')
    
    # Special features
    df['after_midnight'] = 0
    df.loc[df.DepTime > 240, 'after_midnight'] = 1
    
    return df

In [24]:
X_train.head()

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance,route
46343,c-8,c-22,c-1,1805,AA,DFW,LAX,1235,DFWLAX
75701,c-2,c-15,c-3,819,US,DCA,RSW,892,DCARSW
89227,c-10,c-18,c-3,726,CO,BOS,EWR,200,BOSEWR
70825,c-4,c-10,c-7,1515,AA,MCI,ORD,403,MCIORD
26022,c-11,c-24,c-5,1453,YV,MDT,IAD,94,MDTIAD


# Select encoding

## Prepare folds

In [25]:
lgbm_params = {'boosting': 'gbdt',
               'bagging_freq': 1,
               'n_estimators': 10000,
               'random_state': RANDOM_STATE,
               'silent': True,
               'bagging_fraction': 0.638782643397927,
               'feature_fraction': 0.5072049472753989,
               'learning_rate': 0.02238858415024258,
               'max_depth': 10,
               'min_data_in_leaf': 7,
               'reg_alpha': 4.16689412832005,
               'reg_lambda': 3.0220290348080594}

In [26]:
cv_splitter = StratifiedKFold(shuffle=True, random_state=RANDOM_STATE)

In [27]:
%%time
aucs_dict = {name: [] for name in list(target_encoders) + ['ordinal']}
for train_ind, test_ind in cv_splitter.split(X, y):
    print('New fold started')
    X_train, y_train = X.iloc[train_ind], y.iloc[train_ind]
    X_test, y_test = X.iloc[test_ind], y.iloc[test_ind]

    cat_train = X_train[cat_cols]
    cat_test = X_test[cat_cols]

    for name, encoder in target_encoders.items():
        print(f'{name} started')
        enc = encoder.fit(cat_train, y_train)
        X_train = preproc(X_train, enc)
        X_test = preproc(X_test, enc)
        X_tr, X_es, y_tr, y_es = train_test_split(X_train.drop(cat_cols,
                                                               axis=1),
                                                  y_train,
                                                  test_size=0.1,
                                                  shuffle=True,
                                                  random_state=RANDOM_STATE)

        model = lgb.LGBMClassifier()
        model.set_params(**lgbm_params)

        model.fit(X_tr, y_tr,
                  eval_set=[(X_tr, y_tr), (X_es, y_es)],
                  eval_metric=['auc'],
                  early_stopping_rounds=100, verbose=False)
        auc = roc_auc_score(y_test,
                            model.predict_proba(X_test.drop(cat_cols,
                                                            axis=1))[:, 1])
        aucs_dict[name].append(auc)

    enc = OrdinalEncoder(cols=cat_cols,
                         handle_missing='return_nan',
                         handle_unknown='return_nan')
    enc.fit(cat_train)
    X_train = preproc(X_train, enc)
    X_train[cat_cols + '_enc'] = X_train[cat_cols].astype('category')
    X_test = preproc(X_test, enc)
    X_test[cat_cols + '_enc'] = X_test[cat_cols].astype('category')
    X_tr, X_es, y_tr, y_es = train_test_split(X_train.drop(cat_cols,
                                                           axis=1),
                                              y_train,
                                              test_size=0.1,
                                              shuffle=True,
                                              random_state=RANDOM_STATE)

    model = lgb.LGBMClassifier()
    model.set_params(**lgbm_params)
    model.fit(X_tr, y_tr,
              eval_set=[(X_tr, y_tr), (X_es, y_es)],
              eval_metric=['auc'],
              early_stopping_rounds=100, verbose=False)
    auc = roc_auc_score(y_test,
                        model.predict_proba(X_test.drop(cat_cols,
                                                        axis=1))[:, 1])
    aucs_dict['ordinal'].append(auc)

New fold started
js started


/Users/aapiskotin/opt/anaconda3/lib/python3.8/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
<ipython-input-12-0555f0c157ea>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['hour'] = df.DepTime // 100
<ipython-input-12-0555f0c157ea>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['minute'] = df.DepTime % 100
<ipython-input-12-0555f0c157ea>:4: 

[LightGBM] [Warning] boosting is set=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] feature_fraction is set=0.5072049472753989, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5072049472753989
[LightGBM] [Warning] bagging_fraction is set=0.638782643397927, subsample=1.0 will be ignored. Current value: bagging_fraction=0.638782643397927
[LightGBM] [Warning] min_data_in_leaf is set=7, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=7
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
loo started


/Users/aapiskotin/opt/anaconda3/lib/python3.8/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
<ipython-input-12-0555f0c157ea>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['hour'] = df.DepTime // 100
<ipython-input-12-0555f0c157ea>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['minute'] = df.DepTime % 100
<ipython-input-12-0555f0c157ea>:4: 

[LightGBM] [Warning] boosting is set=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] feature_fraction is set=0.5072049472753989, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5072049472753989
[LightGBM] [Warning] bagging_fraction is set=0.638782643397927, subsample=1.0 will be ignored. Current value: bagging_fraction=0.638782643397927
[LightGBM] [Warning] min_data_in_leaf is set=7, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=7
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
target started


...

[LightGBM] [Warning] boosting is set=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] feature_fraction is set=0.5072049472753989, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5072049472753989
[LightGBM] [Warning] bagging_fraction is set=0.638782643397927, subsample=1.0 will be ignored. Current value: bagging_fraction=0.638782643397927
[LightGBM] [Warning] min_data_in_leaf is set=7, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=7
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).


/Users/aapiskotin/opt/anaconda3/lib/python3.8/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
<ipython-input-12-0555f0c157ea>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['hour'] = df.DepTime // 100
<ipython-input-12-0555f0c157ea>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['minute'] = df.DepTime % 100
<ipython-input-12-0555f0c157ea>:4: 

[LightGBM] [Warning] boosting is set=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Warning] min_data_in_leaf is set=7, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=7
[LightGBM] [Warning] feature_fraction is set=0.5072049472753989, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5072049472753989
[LightGBM] [Warning] bagging_fraction is set=0.638782643397927, subsample=1.0 will be ignored. Current value: bagging_fraction=0.638782643397927
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).


/Users/aapiskotin/opt/anaconda3/lib/python3.8/site-packages/lightgbm/basic.py:1286: UserWarning: Overriding the parameters from Reference Dataset.
  warnings.warn('Overriding the parameters from Reference Dataset.')
/Users/aapiskotin/opt/anaconda3/lib/python3.8/site-packages/lightgbm/basic.py:1098: UserWarning: categorical_column in param dict is overridden.
  warnings.warn('{} in param dict is overridden.'.format(cat_alias))


CPU times: user 21min 5s, sys: 35.7 s, total: 21min 41s
Wall time: 6min 58s


In [28]:
X.shape

(100000, 9)

In [38]:
{name: sum(aucs) / 5 for name, aucs in aucs_dict.items()}

{'js': 0.7087563351442323,
 'loo': 0.7086192485232723,
 'target': 0.7094940568958219,
 'woe': 0.7195605741929441,
 'ordinal': 0.7324608373059404}

# Commit

## Refit on full dataset

### Prepare data

In [29]:
train = pd.read_csv(data_path.joinpath('flight_delays_train.csv'))
test = pd.read_csv(data_path.joinpath('flight_delays_test.csv'))

In [30]:
train['route'] = train.Origin + train.Dest

test['route'] = test.Origin + test.Dest

train[target_name] = train[target_name].map({"Y": 1, "N": 0})

In [31]:
X, y = train.drop(target_name, axis=1), train[target_name]

In [32]:
%%time
for name, encoder in target_encoders.items():
    encoder.fit(X[cat_cols], y)
    print(f'{name} finished!')

/Users/aapiskotin/opt/anaconda3/lib/python3.8/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


js finished!
loo finished!


/Users/aapiskotin/opt/anaconda3/lib/python3.8/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


target finished!


/Users/aapiskotin/opt/anaconda3/lib/python3.8/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


woe finished!
CPU times: user 1.66 s, sys: 130 ms, total: 1.79 s
Wall time: 2.01 s


In [33]:
enc = OrdinalEncoder(cols=cat_cols,
                     handle_missing='return_nan',
                     handle_unknown='return_nan')

In [34]:
enc.fit(X[cat_cols]);

/Users/aapiskotin/opt/anaconda3/lib/python3.8/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


In [35]:
X = preproc(X)

TypeError: preproc() missing 1 required positional argument: 'enc'

In [ ]:
X_tr, X_es, y_tr, y_es = train_test_split(X, y,
                                          test_size=0.1,
                                          shuffle=True,
                                          random_state=RANDOM_STATE)

In [ ]:
%%time
model = lgb.LGBMClassifier()
model.set_params(**lgbm_params)

model.fit(X_tr, y_tr,
          eval_set=[(X_tr, y_tr), (X_es, y_es)],
          eval_metric=['auc'],
          early_stopping_rounds=100)

## Prepare test

In [ ]:
test = preproc(test)

In [ ]:
test['dep_delayed_15min'] = model.predict_proba(test)[:, 1]

In [ ]:
test['dep_delayed_15min'].to_csv(data_path.joinpath('submit.csv'), index_label='id')

# Next Steps

* Grid Search through params
* datetime engineering (sin/cos transform, DepTime limits transform(?))

# Notes

* OHE sucks since model just doesn't use those features
* there are only 1 after 2400

__Best Score__: 0.72329